# PDH Feedback Control

This notebook is essentially a paraphrasing of [this document](../Resources/pdh_control.pdf) along with some additional justifications to explain why some things are the way they are. The paper should be used as the validation mechanism if anything written here seems off. To represent imaginary numbers j is used instead of i now because current is now a thing.


## 1.1: Basic Concepts

Define the following terms:

$$
\hat{A}(s) = \mathcal{L}\{h(t)\} = \frac{Y(s)}{X(s)}
$$

Where $s = \sigma + 2\pi f = \sigma + j\omega$. Because we are analyzing the closed-loop behaviour of the system we set $\sigma$ equal to 0. We can do this because it's a closed loop, so things like an unstable impulse response where a positive $\sigma$ would cause the system output to become unstable is already encoded in the closed-loop transfer function evaluated along the imaginary axis. In other words, we can work purely in the frequency domain when evaluating the closed-loop behaviour of this system.

The gain and phase of $\hat{A}$ are defined as follows:

$$
|\hat{A}|, \angle{\hat{A}}
$$


Loop components perform algebraic operations on their inputs in the frequency domain. These operations correspond to convolutions in the time domain. The open-loop transfer function is the product of all loop components:

$$
\alpha = \hat{K}\hat{G}\hat{H}
$$

In the time domain this represents the end-to-end convolution applied to the input signal. We use a negative feedback loop because we need a way to self-regulate our current response as a function of our priors. Assuming the last output is fed as negative feedback to be subtracted from our input signal before passing through the chain, we can define the closed-loop transfer function as follows:

$$
e = x - y\newline
y = \alpha e = \alpha(x - y)\newline
(1 + \alpha)y = \alpha x\newline
\frac{y}{x} = \frac{\alpha}{1 + \alpha}
$$

This is super unintuitive unless you treat $y_{o, n}$ and $y_{e, n-1}$ as the same term $y$. We can do this because our system is assumed to be linear time-invariant (LTI). We make this assumption because our transfer function does not change with respect to time. By definition of what the laplace transform does, no frequency domain signals will change with respect to time. This means niether X(s), Y(s), nor H(s) will change with respect to whatever cycle of the loop we are on. It also makes the concept of loop cycles irrelevant because doing this in the frequency domain is inherently equivalent to integrating this recursive feedback loop across all of time. This means that our feedback loop modelled in the laplace domain really just models the convergence of a series in the time domain where each term in that series is a convolution between our transfer function and our input.

Let $h(t)$ represent the open-loop impulse response such that a single pass $y_{\text{open}}(t) = (h * x)(t)$


$$
y_0(t) = 0\newline
e_1(t) = x(t) - y_0(t) = x(t)\newline
y_1(t) = (h * e_1)(t) = (h * x)(t)\newline
e_2(t) = x(t) - y_1(t)\newline
y_2(t) = (h * e_2)(t) = (h * x)(t) - (h * h * x)(t)\newline
e_3(t) = x(t) - y_2(t)\newline
y_3(t) = (h * x)(t) - (h * h * x)(t) + (h * h * h * x)(t)\newline
$$


Through induction we get the following series:

$$
y_n(t) = \sum_{k=1}^n (-1)^{k-1}(h^{*k} * x)(t)
$$

The steady-state solution is the limit:

$$
y(t) = \lim_{n\rightarrow\infty}y_n(t) = \sum_{k=1}^\infty (-1)^{k-1}(h^{*k} * x)(t)\newline
$$

We can also map this series to its Laplace domain counterpart:

$$
\mathcal{L}\{h^{*k}(t)\} = [H(s)]^k\newline
Y(s) = \sum_{k=1}^\infty(-1)^{k-1}H(s)^kX(s) = \frac{H(s)}{1 + H(s)}X(s)
$$

The Laplace domain representation gives us a geometric series that converges for all $|H(s)| < 1$. In other words, it will converge as long as the open-loop transfer function does not boost the signal. The frequency domain representation is the sum of the infinite series of convolutions that represent the limiting behaviour of our feedback process. 

Real-world systems have a delay between the input and the output. We can introduce $\tau$-shifted delta convolutions into each step of our series to account for this:

$$
h_\tau = h(t) * \delta(t-\tau)
y_n(t) = \sum_{k=1}^n(-1)^{k-1}(h * h_\tau^{*(k-1)} * x)(t)\newline

$$

Via $\mathcal{L}\{\delta(t - \tau)\} = e^{-s\tau}$

$$
Y(s) = H(s)X(s)\sum_{k=0}^{\infty}(-1)^kH^k(s)e^{-s\tau k}
$$

If $|H(s)e^{-s\tau}| < 1$, this geometric series converges to
 
$$
Y(s) = \frac{H(s)}{1 + H(s)e^{-s\tau}}X(s)
$$

The stability of this closed-loop expression is determined by the roots of $1 + H(s)e^{-s\tau} = 0$.


This was a bit off-topic and rather tedious, but I think it's much more powerful and rewarding to think of control theory as a form of applied recursion rather than just memorizing a bunch of block diagram hacks to be implemented through black-box Matlab code.



## 1.2: Basic Model

Now we have the foundation to build up our control model a little bit. This is what it looks like:

#### Figure 1.2: System Diagram and Bode Plots
<p align="left">
  <img src="../images/control_1.png" width="500">
</p>

In the context of figure b we get:

$$
\alpha = KGH
$$
$$
\frac{y_5}{m_6} = \frac{\alpha}{1 + \alpha}
$$
$$
y_1 = \frac{\alpha}{1 + \alpha}(\frac{m_6}{H} + \frac{n_1}{\alpha} - \frac{n_5}{H})
$$

We can correlate a signal with a delayed version of itself to figure out how much power the signal shares with a time-shifted version of itself. This is called autocorrelation. We can take the Fourier transform of that autocorrelation across all given time offsets to get the power spectral density (PSD) function which tells us how much of the signal's power is being dispersed at any given frequency.

$$
S_{xx}(f) = \int_{-\infty}^\infty R_{xx}(\tau)e^{-j2\pi f\tau}d\tau
$$

Where
$$
R_{xx}(\tau) = \lim_{T\rightarrow\infty}\frac{1}{T}\int_0^Tx(t)x(t + \tau)dt
$$

Then 
$$
S_{y1} = \frac{1}{|1 + \alpha|^2}S_{n1} + \frac{|\alpha|^2}{|1 + \alpha|^2}\frac{1}{|H|^2}S_{n5}
$$

Ideal negative feedback is defined by unity closed-loop gain at frequencies of interest i.e. 

$$
|\alpha(j\omega)| \rightarrow \infty
$$
$$
\angle \alpha(j\omega) = 0\newline
$$
$$
\frac{\alpha}{1 + \alpha} \rightarrow 1
$$

This implicitly means that noise from $n_1$ is completely eliminated. However, sensor noise from $n_5$ is still present:
$$
S_{y1} \rightarrow \frac{1}{|H|^2}S_{n5}
$$

From the above, we see that high gains at frequencies of interest will suppress laser noise but at the expense of increased susceptibility to sensor noise. Intuitively, as we increase the gain we are better able to detect laser noise, and thus better able to apply feedback to correct it. However, as our gain increases, so does the amount of noise introduced by the sensor because it is getting boosted by the closed-loop gain as well, and we are not able to correct it with feedback. This means that we need to consider the tradeoff between plant (laser) and sensor noise during the design process.

As per standard convention, we define the following terms unity gain as the frequency where $|\alpha(f_{UG})| = 1 = 0dB$ and the phase margin as $\phi_m = \pi + \angle\alpha(f_{UG})$. Phase margin tells us how much room we have at unity gain before our system becomes unstable. Given the characteristic equation:

$$
1 + \alpha = 0
$$
$$
|\alpha| = 1
$$

It's easy to see $\angle\alpha = \pi$ at unity gain would cause a pole. Phase margin is simply a way to quantify how far we are from this pole. We can also do the inverse of this by getting the frequency where phase is equal to pi, then figuring out the gain margin by dividing 1 by the gain at that frequency i.e. $\angle \alpha(f_\pi) = -\pi \text{ and } g_m = \frac{1}{|\alpha(f_\pi)|}$. 

Let $e = x - y = x - \alpha e \rightarrow \frac{e}{x} = \frac{1}{1 + \alpha}$. Around unity gain we see a bump in this error response because this is when the magnitude of the error response denominator tends to be smallest. We define $f_{bump} = \argmax_\omega|\frac{E(j\omega)}{X(j\omega)}| = \argmax_\omega|\frac{1}{1 + \alpha}|$. This maps to the frequency with the highest PSD on the error response's spectrum and it represents the frequency at which the system is worst at tracking disturbances. 

Controllers are made to minimize error, the point where the error response is highest is the point where the controller is doing the worst job over the course of the recursion's life cycle at minimizing error. In other words, modes within the loop operating at this frequency will contribute disproportionately to the overall error present throughout controller's operation.

What I mean by this is that a control loop is a recursion. Modes of different frequencies propagate through that recursion with respect to time. Error accumulates with respect to time as the controller is operating. Transforming to the frequency domain collapses that recursion. Modes oscillating at the servo bump frequency contribute to the overall recursion's lifetime error the most. 

System stability can be throught of as the interference patterns between modes in the system being bounded such that there exists an upper limit on their magnitude that is within the system's tolerances. As a result of asserting this bound, there is virtually never a good time to use positive feedback in a control loop as recursion with positive feedback yields to unbounded behaviour. This generally results in either damage or saturation of the system, in both cases the response tends to be highly inappropriate. This intuition is formalized through the concepts of BIBO stability and the Nyquist criterion. [Lyapunov Stability](https://www.cds.caltech.edu/~murray/courses/cds101/fa02/caltech/mls93-lyap.pdf) generalizes this further by defining a generic energy function from which to evalutate the system's stability.

We showed the loop above but will now solidify the context. We define our plant as $G(f)$, this represents our laser. Our sensor is represented as $H(f)$. This gives the Laplace domain representation of our error signal with respect to the laser output as per [the preceding notebook](../notebooks/pound_drever_hall.ipynb). This assumes that we have a Laplace model for our laser. If we don't then we should seek to find the black-box relationship between the error signal and the laser driver current instead. Assume that H includes the mixer and LPF as discussed previously. Analog PID control can be implemented by a loop filter where it's transfer function can be written as:

$$
K = K_p(1 + \frac{\omega_I}{j\omega} + j\frac{\omega}{\omega_D}) = K_p(1 - j\frac{\omega_I}{\omega} + j\frac{\omega}{\omega_D})
$$

Where $\omega_I$ and $\omega_D$ are the integral and derivative corner frequencies respectively. The reason why $j\omega$ maps to differentiation in the time domain or $\frac{-j}{\omega}$ maps to integration in the time domain is because when they are applied to a signal in the frequency domain, their representation is equivalent to if that frequency signal's native time-domain counterpart was transformed with either a differentiated or integrated Fourier operator respectively. By integration by parts, we can see that this is in turn equivalent to a standard Fourier operator transforming the differentiated or integrated signal (respectively) instead of the native signal.



## 1.3: Convergence Game

The main idea behind everything we are doing here is that we are building something to set the state of some given system to target state as quickly and as predictably as possible. We do this by using a feedback loop. This loop is inherently recursive, and the convergence of this recursion is the convergence of our actual state with our target state. If the recursion diverges so does our state. We use frequency domain analysis so we can collapse the recursion and reason about it algebraically. We then bring in frequency-domain representations of each part of our system to do this. Once we have a closed-loop representation of our system, we can get to work mapping the frequency spectra of all possible input states. This representation, along with the frequency representations of the injected noise sources, allows us to model all modes that will oscillate throughout our recursion. We can then perform stability analysis both algebraically and numerically to assess how our system responds to these oscillations. We can measure how effectively our system converges by how small the error response is. The smaller the error response, the less overall error in the system and the smaller the difference between the curves of the target and the actual state plotted in the time domain. We can tweak our system by changing the physical things in it that impact the transfer function of each block in our feedback loop e.g. resistors, capacitors etc. By doing this, we are able to implement recursion in a useful manner through a physical system. This is reasonably cool.

Generally speaking, there are three key ways to make the system more effective:

1. Higher performance (Lower error)

2. Higher bandwidth (Faster speed)

3. Higher robustness (More stable)

Higher performance is ususally assesed by the magnitude characteristics of the error response. Higher bandwidth means that the feedback loop is better able to track fast-changing signals. Higher robustness means the system is less likely to become unstable. We can generally optimize pretty well for two of these things at the same time but we sacrifice the third. As such, it makes the most sense to define a minimal baseline level of robustness and then optimize everything else from there. This is what we will now do:

1. Maximize $f_{UG}$. Unity gain is where our closed-loop response starts to fall apart. In other words, it marks the boundary between good control and poor control. By making this as large as possible, we increase the effective bandwidth of our system allowing us to better use higher frequency components in our system. 

2. Maintain $\frac{\pi}{6} < \phi_m < \frac{\pi}{3}$. A better margin would be nice for more stabillity but it comes at the cost of a slower system. Remember, $\mathcal{F}\{\delta(t-\tau) = e^{-j\omega\tau}\}$. Every time we wind our response's phase back in the frequency domain we impart a delay in the time domain. You can also think of this as pulses travelling around a loop. If the loop is too fast, the next pulse will reach a node where the last pulse still hasn't finished dissipating. If this happens the pulses will stack, and then that pulse will be fed through the loop again back onto itself, which still hasn't finished dissipating. This is in essense positive feedback. And it's described in the frequency domain by a phase shift of $\pi$. However, it's a lot more natural to think about it in the time domain first before trying to come to terms with the frequency domain expression for it.

3. For $f < f_{UG}$, maintain $\angle\alpha > -\frac{2}{3}\pi$. This gives us a nice stability margin to ensure no poles are present in our active range.